# DL Tutorial Day 13

## Implementing a Neural Network from scratch

We want to create a class that will mimic the functionality of the keras neural network class.

In [290]:
import pandas as pd
from sklearn.model_selection import train_test_split as tts
import numpy as np

In [291]:
# importing data
df = pd.read_csv('insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [292]:
# preparing the data
X_train, X_test, y_train, y_test = tts(df[['age','affordibility']], df['bought_insurance'], test_size = 0.2)
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age']/100
X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age']/100

X_test_scaled

,age,affordibility
15,0.55,1
12,0.27,0
8,0.62,1
18,0.19,0
5,0.56,1
17,0.58,1


In [293]:
class myNN:

    # constructor
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0

    # loss function
    def logloss(self, yt, yp):
        epsilon = 1e-15
        yp = np.array([max(i, epsilon) for i in yp])
        yp = np.array([min(i, 1-epsilon) for i in yp])
        loss = -np.mean(yt*np.log(yp) + (1-yt)*np.log(1-yp))
        return loss

    # activation function
    def sigmoid_numpy(self, x):
        return np.array(1/(1+np.exp(-x)))
    
    # defining the fit method
    def fit(self, X, y, learning_rate, epochs):
        self.w1, self.w2, self.bias = self.gradient_descent(X['age'], X['affordibility'], y, learning_rate, epochs)
    
    # prediction function
    def prediction_function(self, X_test):
        weighted_sum = X_test['age']*self.w1 + X_test['affordibility']*self.w2 + self.bias
        probability = self.sigmoid_numpy(weighted_sum)
        prediction = [1 if i>=0.5 else 0 for i in probability]
        return prediction

    # gradient descent algorithm
    def gradient_descent(self, age, affordability, y_true, learning_rate, epochs, loss_threshold = 0.5):
        w1 = w2 = 1
        bias = 0
        n = len(age)

        # implementing the loop until loss falls below the threshold
        for i in range(epochs):
            weighted_sum = w1*age + w2*affordability + bias
            y_predicted = self.sigmoid_numpy(weighted_sum)

            loss = self.logloss(y_true, y_predicted)

            w1_derivative = (1/n)*np.dot(np.transpose(age), (y_predicted-y_true))
            w2_derivative = (1/n)*np.dot(np.transpose(affordability), (y_predicted-y_true))
            bias_derivative = np.mean(y_predicted-y_true)

            w1 = w1 - learning_rate*w1_derivative
            w2 = w2 - learning_rate*w2_derivative
            bias = bias - learning_rate*bias_derivative

            if i%100 == 0:
                print(f"Epoch: {i+1} W1: {w1} W2: {w2} Bias: {bias} Loss: {loss}")

            if loss < loss_threshold:
                print("Solution converged!")
                break
        return w1, w2, bias


In [294]:
customModel = myNN()
customModel.fit(X_train_scaled, y_train, 0.001, 20000)

Epoch: 1 W1: 0.9999333655766773 W2: 0.9998196654374881 Bias: -0.00027887661205984384 Loss: 0.8029913001994052
Epoch: 101 W1: 0.9934141649567464 W2: 0.9820539298006897 Bias: -0.02778245707287551 Loss: 0.7918400092009024
Epoch: 201 W1: 0.9871791026858434 W2: 0.9648176709010559 Bias: -0.054529762134414676 Loss: 0.7813207550419676
Epoch: 301 W1: 0.9812255855129265 W2: 0.9481090252840213 Bias: -0.08052815248059784 Loss: 0.7714101268666528
Epoch: 401 W1: 0.9755505504343607 W2: 0.9319250101255144 Bias: -0.10578621559674432 Loss: 0.7620841103510653
Epoch: 501 W1: 0.970150496122459 W2: 0.9162615818368861 Bias: -0.13031367974043012 Loss: 0.7533182768287282
Epoch: 601 W1: 0.965021516421812 W2: 0.9011137012454824 Bias: -0.15412132283056237 Loss: 0.7450879636417802
Epoch: 701 W1: 0.9601593353229773 W2: 0.8864754039176106 Bias: -0.17722087783198098 Loss: 0.7373684434722896
Epoch: 801 W1: 0.9555593428451117 W2: 0.8723398742192794 Bias: -0.19962493614348817 Loss: 0.7301350808884205
Epoch: 901 W1: 0.95

In [295]:
customModel.prediction_function(X_test_scaled)

[1, 0, 1, 0, 1, 1]

In [296]:
y_test

15    1
12    0
8     1
18    0
5     1
17    1
Name: bought_insurance, dtype: int64